<a href="https://colab.research.google.com/github/ViVeK-KuMaR3/GenAI/blob/main/Assignement-2-Text_classification_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
resume_data=load_dataset('Keviv123/resume_dataset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
resume_data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'labels'],
        num_rows: 1738
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'labels'],
        num_rows: 746
    })
})

In [5]:
resume_data['train'][0]

{'Unnamed: 0': 228,
 'text': "         DIRECTOR OF INFORMATION TECHNOLOGY         Professional Profile    Senior Project Management Position Senior Project Manager with over 25 years of diverse experience including health care, private sector, local and state government and aerospace/defense contracting. Successful management of fast-paced private sector projects as well as large multi-departmental/multi-agency government projects. Provided mentoring and professional quality training to hundreds of project managers. Proven competence in leadership, communication, project planning, budgeting, design, change control, execution, implementation and support.          Experience      Director of Information Technology  ,   10/2013   to   Current    Company Name          Member of the Health Care Executive team responsible for the delivery of technology to two hospitals, 40 clinics, Public Health, Behavioral Health, Medical Examiner and the County's Health Insurance plan.  Implemented Service

In [6]:
resume_data['test'][0]

{'Unnamed: 0': 1626,
 'text': '         SR. MANAGER       Summary     Over twenty-five years Management experience in Vendor Compliance, Product Integrity/Quality, Customer Support/Sales and Business Office Management.  I am a results-focused professional with the ability to manage multiple projects and meet deadlines in a fast paced environment.  Specific expertise includes: quality assurance, strategic planning, handling of large customer accounts, office management, vendor billing and invoicing, purchasing and payroll.       Highlights          Exceptional Customer Service Skills  Exceptional time management skills  Adaptable      Business and requirements analysis            Experience     10/2008   to   07/2012     Sr. Manager    Company Name   －   City  ,   State      Vendor Management and Product Integrity/Quality Responsibilities include:  Partnering with Quality Assurance Team, Sourcing Managers, Buyers, Technical Design and Logistics to ensure packaging and garment labeling m

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True,max_length=512,padding=True)


In [9]:
tokenized_resume = resume_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/1738 [00:00<?, ? examples/s]

Map:   0%|          | 0/746 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
id2label={
 "0": 'AUTOMOBILE',
 "1": 'BANKING',
 "2": 'ADVOCATE',
 "3": 'FINANCE',
 "4": 'CONSULTANT',
 "5": 'INFORMATION-TECHNOLOGY',
 "6": 'SALES',
 "7": 'AGRICULTURE',
 "8": 'ACCOUNTANT',
 "9": 'BUSINESS-DEVELOPMENT',
 "10": 'PUBLIC-RELATIONS',
 "11": 'HR',
 "12": 'DIGITAL-MEDIA',
 "13": 'AVIATION',
 "14": 'ARTS',
 "15": 'CONSTRUCTION',
 "16": 'TEACHER',
 "17": 'CHEF',
 "18": 'FITNESS',
 "19": 'DESIGNER',
 "20": 'APPAREL',
 "21": 'ENGINEERING',
 "22": 'BPO',
 "23": 'HEALTHCARE'}

In [14]:
label2id={
  'AUTOMOBILE': "0",
 'BANKING': "1",
 'ADVOCATE': "2",
 'FINANCE': "3",
 'CONSULTANT': "4",
 'INFORMATION-TECHNOLOGY': "5",
 'SALES': "6",
 'AGRICULTURE': "7",
 'ACCOUNTANT': "8",
 'BUSINESS-DEVELOPMENT': "9",
 'PUBLIC-RELATIONS': "10",
 'HR': "11",
 'DIGITAL-MEDIA': "12",
 'AVIATION': "13",
 'ARTS': "14",
 'CONSTRUCTION': "15",
 'TEACHER': "16",
 'CHEF': "17",
 'FITNESS': "18",
 'DESIGNER': "19",
 'APPAREL': "20",
 'ENGINEERING': "21",
 'BPO': "22",
 'HEALTHCARE': "23"

 }


In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

resume_classification_model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=24, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="resume_classification_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=resume_classification_model,
    args=training_args,
    train_dataset=tokenized_resume["train"],
    eval_dataset=tokenized_resume["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.619870,0.596515
2,No log,1.815423,0.733244
3,No log,1.613308,0.753351


TrainOutput(global_step=327, training_loss=2.2574879124259364, metrics={'train_runtime': 99.166, 'train_samples_per_second': 52.578, 'train_steps_per_second': 3.298, 'total_flos': 690955999100928.0, 'train_loss': 2.2574879124259364, 'epoch': 3.0})

In [17]:
resume_classification_model.save_pretrained('resume-classification')

In [19]:
tokenizer.save_pretrained('resume-tokenizer')

('resume-tokenizer/tokenizer_config.json',
 'resume-tokenizer/special_tokens_map.json',
 'resume-tokenizer/vocab.txt',
 'resume-tokenizer/added_tokens.json',
 'resume-tokenizer/tokenizer.json')

#INFERENCING

In [20]:
testing_tokenizer=AutoTokenizer.from_pretrained('/content/resume-tokenizer')

In [21]:
loaded_model=AutoModelForSequenceClassification.from_pretrained('/content/resume-classification')

In [39]:
resume='''flight hours is 7000hrs till now'''

In [40]:
resume

'flight hours is 7000hrs till now'

In [41]:
from transformers import pipeline

classifier=pipeline('text-classification',model=loaded_model,tokenizer=testing_tokenizer,max_length=512, truncation=True)

In [42]:
classifier(resume)

[{'label': 'AVIATION', 'score': 0.08167459815740585}]